# Our plan

We want to try to build up some functionality to create etl pipelines for pandas operations.
We want to define classic extract, transform and load functions which we can string together in a pipeline. 
We would also like to add some functionality to inspect the intermediate dataframes and some key properties.

## Our background

We work as Data Science Consultants mainly with SME's in manufacturing. We often have to build up etl pipelines from various sources to then use for analytics or ML workloads.
The pipelines can get pretty complex with custom business logic, that why we want to have a way to quickly show all the steps we take.

In [ ]:
from dialoghelper import *
from fastcore.tools import *

Tools available from `dialoghelper`:

- &`curr_dialog`: Get the current dialog info.
- &`msg_idx`: Get absolute index of message in dialog.
- &`add_html`: Send HTML to the browser to be swapped into the DOM using hx-swap-oob.
- &`find_msg_id`: Get the current message id.
- &`find_msgs`: Find messages in current specific dialog that contain the given information.
  - (solveit can often get this id directly from its context, and will not need to use this if the required information is already available to it.)
- &`read_msg`: Get the message indexed in the current dialog.
  - To get the exact message use `n=0` and `relative=True` together with `msgid`.
  - To get a relative message use `n` (relative position index).
  - To get the nth message use `n` with `relative=False`, e.g `n=0` first message, `n=-1` last message.
- &`del_msg`: Delete a message from the dialog.
- &`add_msg`: Add/update a message to the queue to show after code execution completes.
- &`update_msg`: Update an existing message.
- &`url2note`: Read URL as markdown, and add a note below current message with the result
- &`msg_insert_line`: Insert text at a specific location in a message.
- &`msg_str_replace`: Find and replace text in a message.
- &`msg_strs_replace`: Find and replace multiple strings in a message.
- &`msg_replace_lines`: Replace a range of lines in a message with new content.
  - Always first use `read_msg( msgid=msgid, n=0, relative=True, nums=True)` to view the content with line numbers.

Tools available from `fastcore.tools`:

- &`rg`: Run the `rg` command with the args in `argstr` (no need to backslash escape)
- &`sed`: Run the `sed` command with the args in `argstr` (e.g for reading a section of a file)
- &`view`: View directory or file contents with optional line range and numbers
- &`create`: Creates a new file with the given content at the specified path
- &`insert`: Insert new_str at specified line number
- &`str_replace`: Replace first occurrence of old_str with new_str in file
- &`strs_replace`: Replace for each str pair in old_strs,new_strs
- &`replace_lines`: Replace lines in file using start and end line-numbers

In [ ]:
import pandas as pd

# Sample manufacturing data
data = {
    'order_id': [101, 102, 103, 104, 105],
    'product': ['Widget A', 'Widget B', 'Widget A', 'Widget C', 'Widget B'],
    'quantity': [50, 30, 75, 20, 45],
    'defects': [2, 1, 3, 0, 2],
    'production_time': [120, 95, 150, 80, 110]  # in minutes
}

df = pd.DataFrame(data)
df

,order_id,product,quantity,defects,production_time
0,101,Widget A,50,2,120
1,102,Widget B,30,1,95
2,103,Widget A,75,3,150
3,104,Widget C,20,0,80
4,105,Widget B,45,2,110


In [ ]:
df[df["product"] != "Widget C"].groupby(["product"])[["quantity", "defects"	,"production_time"]].sum()

,quantity,defects,production_time
product,,,
Widget A,125,5,270
Widget B,75,3,205


Say we have a pipeline like this. What are some approaches to implement our plan?

##### 🤖Reply🤖<!-- SOLVEIT_SEPARATOR_7f3a9b2c -->

Great example! I can see you're filtering and then aggregating. For implementing your ETL pipeline plan with inspection capabilities, here are a few approaches:

1. **Function chaining with a custom Pipeline class** - Each step returns a modified dataframe, and the class stores intermediate results for inspection

2. **Decorator-based approach** - Wrap your transform functions with decorators that automatically log/store dataframe properties at each step

3. **Simple function list with a runner** - Keep a list of functions and a separate runner that executes them while capturing snapshots

Which approach sounds most appealing for your workflow? Or would you like me to briefly explain any of these in more detail before you decide?

Show a simple example for each please

##### 🤖Reply🤖<!-- SOLVEIT_SEPARATOR_7f3a9b2c -->

I'll show you a simple example skeleton for each approach. Since you asked explicitly, I'll write out the code examples:

**1. Pipeline class approach:**
```python
class ETLPipeline:
    def __init__(self, df):
        self.steps = []
        self.df = df
        
    def add_step(self, func, name):
        self.df = func(self.df)
        self.steps.append({'name': name, 'shape': self.df.shape})
        return self
    
    def get_result(self):
        return self.df
```

**2. Decorator approach:**
```python
def track_step(step_name):
    def decorator(func):
        def wrapper(df):
            result = func(df)
            print(f"{step_name}: {result.shape}")
            return result
        return wrapper
    return decorator

@track_step("Filter")
def filter_products(df):
    return df[df["product"] != "Widget C"]
```

**3. Function list with runner:**
```python
def run_pipeline(df, steps):
    results = []
    for name, func in steps:
        df = func(df)
        results.append({'step': name, 'shape': df.shape})
    return df, results
```

Which of these feels most natural for your workflow? Would you like to try implementing one with your actual pipeline example?

How would we deal with methods vs functions in each approach?

##### 🤖Reply🤖<!-- SOLVEIT_SEPARATOR_7f3a9b2c -->

Good question! Methods vs functions is an important consideration for each approach.

**Pipeline class**: Works naturally with both - you can pass either functions or methods as long as they take a dataframe and return a dataframe. Methods would need to be bound or wrapped.

**Decorator**: Best suited for functions. For methods, you'd need to handle `self` carefully in the wrapper, which adds complexity.

**Function list runner**: Similar to the pipeline class - works with both as long as they're callable and follow the df-in, df-out pattern.

The key question is: are you planning to use pandas methods (like `.groupby()`, `.merge()`) or write your own functions that wrap those operations? This might influence which approach fits best.

What's your typical style - do you prefer chaining pandas methods or writing custom transform functions?

Can you write our small example transformation for each approach

##### 🤖Reply🤖<!-- SOLVEIT_SEPARATOR_7f3a9b2c -->

Since you've explicitly asked me to write the code, I'll create the full example for each approach using your filter + groupby transformation:

**1. Pipeline class approach:**
```python
class ETLPipeline:
    def __init__(self, df):
        self.steps = []
        self.df = df
        self.history = [{'step': 'initial', 'shape': df.shape, 'columns': list(df.columns)}]
        
    def add_step(self, func, name):
        self.df = func(self.df)
        self.history.append({'step': name, 'shape': self.df.shape, 'columns': list(self.df.columns)})
        return self
    
    def get_result(self):
        return self.df
    
    def show_history(self):
        return pd.DataFrame(self.history)

# Usage
pipeline = ETLPipeline(df)
result = (pipeline
    .add_step(lambda d: d[d["product"] != "Widget C"], "filter_widget_c")
    .add_step(lambda d: d.groupby(["product"])[["quantity", "defects", "production_time"]].sum(), "aggregate")
    .get_result())

print(pipeline.show_history())
print("\nResult:")
print(result)
```

**2. Decorator approach:**
```python
steps_log = []

def track_step(step_name):
    def decorator(func):
        def wrapper(df):
            result = func(df)
            steps_log.append({'step': step_name, 'shape': result.shape, 'columns': list(result.columns)})
            return result
        return wrapper
    return decorator

@track_step("filter_widget_c")
def filter_products(df):
    return df[df["product"] != "Widget C"]

@track_step("aggregate")
def aggregate_by_product(df):
    return df.groupby(["product"])[["quantity", "defects", "production_time"]].sum()

# Usage
result = aggregate_by_product(filter_products(df))
print(pd.DataFrame(steps_log))
print("\nResult:")
print(result)
```

**3. Function list with runner:**
```python
def run_pipeline(df, steps):
    history = [{'step': 'initial', 'shape': df.shape, 'columns': list(df.columns)}]
    current_df = df
    
    for name, func in steps:
        current_df = func(current_df)
        history.append({'step': name, 'shape': current_df.shape, 'columns': list(current_df.columns)})
    
    return current_df, pd.DataFrame(history)

# Define steps
steps = [
    ("filter_widget_c", lambda d: d[d["product"] != "Widget C"]),
    ("aggregate", lambda d: d.groupby(["product"])[["quantity", "defects", "production_time"]].sum())
]

# Usage
result, history = run_pipeline(df, steps)
print(history)
print("\nResult:")
print(result)
```

Try running these! Which approach feels most intuitive for your workflow?

So we are thinking of a hybrid of 2 and 3). So we would write a decorator that adds a logging keyword to each function, then we can determine for each function in the pipeline what to log/how much to log. We want to keep it as simple as possible. The pipeline should just take a df and a list of functions as steps of the pipeline.

In [ ]:
def pipeline(df, steps, vrbs_default=True):
    for func, func_kwargs in steps:
        vrbs =  func_kwargs.get("vrbs", vrbs_default)
        func_kwargs.update({"vrbs": vrbs})
        df = func(df, **func_kwargs)
    return df

In [ ]:
def track(func):
    def wrapper(in_df, vrbs=False, *args, **kwargs):
        if vrbs:
            print(f"\n{'*'*10} Step: {func.__name__} {'*'*10}")
            print(f"\nInput DataFrame shape: {in_df.shape}")
            print(in_df)
        out_df = func(in_df, *args, **kwargs)
        if vrbs:
            print(f"\nOutput DataFrame shape: {out_df.shape}")
            print(out_df)
        return out_df
    return wrapper

In [ ]:
@track
def filter_products(df):
    return df[df["product"] != "Widget C"]

@track
def aggregate_by_product(df):
    return df.groupby(["product"])[["quantity", "defects", "production_time"]].sum()

We include a way to handle pipeline overwrites of the `vrbs` argument for specific transformation steps. In this example, no information for `aggregate_by_product` is printed:

In [ ]:
steps = [
    (filter_products, {}),
    (aggregate_by_product, {"vrbs": False}),
]

In [ ]:
pipeline(df, steps)


********** Step: filter_products **********

Input DataFrame shape: (5, 5)
   order_id   product  quantity  defects  production_time
0       101  Widget A        50        2              120
1       102  Widget B        30        1               95
2       103  Widget A        75        3              150
3       104  Widget C        20        0               80
4       105  Widget B        45        2              110

Output DataFrame shape: (4, 5)
   order_id   product  quantity  defects  production_time
0       101  Widget A        50        2              120
1       102  Widget B        30        1               95
2       103  Widget A        75        3              150
4       105  Widget B        45        2              110


,quantity,defects,production_time
product,,,
Widget A,125,5,270
Widget B,75,3,205


We want to add basic profiling of time and memory. 

In [ ]:
from datetime import datetime
from time import sleep

In [ ]:
start_time = datetime.now()
sleep(3.14)
end_time = datetime.now()
total_time = end_time - start_time
total_time

datetime.timedelta(seconds=3, microseconds=140467)

In [ ]:
start_time

datetime.datetime(2025, 11, 18, 17, 5, 39, 805194)

In [ ]:
def track(func):
    def wrapper(in_df, vrbs=False, *args, **kwargs):
        start_time = datetime.now()
        if vrbs:
            
            print(f"\n{'*'*10} Step: {func.__name__} {'*'*10}")
            print(f"\nInput DataFrame shape: {in_df.shape}")
            print(f"Start time: {start_time}")
            print(in_df)
        out_df = func(in_df, *args, **kwargs)
        

        end_time = datetime.now()
        total_time = end_time - start_time
        if vrbs:
            print(f"\nOutput DataFrame shape: {out_df.shape}")
            print(f"End time: {end_time}")
            print(f"Total time: {total_time}")
            print(out_df)
        return out_df
    return wrapper

In [ ]:
@track
def filter_products(df):
    return df[df["product"] != "Widget C"]

@track
def aggregate_by_product(df):
    return df.groupby(["product"])[["quantity", "defects", "production_time"]].sum()

In [ ]:
steps = [
    (filter_products, {}),
    (aggregate_by_product, {}),
]

In [ ]:
_df = pipeline(df, steps)


********** Step: filter_products **********

Input DataFrame shape: (5, 5)
Start time: 2025-11-18 17:08:37.229401
   order_id   product  quantity  defects  production_time
0       101  Widget A        50        2              120
1       102  Widget B        30        1               95
2       103  Widget A        75        3              150
3       104  Widget C        20        0               80
4       105  Widget B        45        2              110

Output DataFrame shape: (4, 5)
End time: 2025-11-18 17:08:37.232130
Total time: 0:00:00.002729
   order_id   product  quantity  defects  production_time
0       101  Widget A        50        2              120
1       102  Widget B        30        1               95
2       103  Widget A        75        3              150
4       105  Widget B        45        2              110

********** Step: aggregate_by_product **********

Input DataFrame shape: (4, 5)
Start time: 2025-11-18 17:08:37.233644
   order_id   product  quantity

- right now we are still printing the whole dataframe. We want to change that to just show the first 3, 5 random and the last 3 rows, maybe with some visual clue to show that the rows are truncated.
- we would also like show the dataframes to the right of the aggregate infos (like shape, etc.)
- include information like df.info()
- include information like df.describe()
- include information like diff columns (num cols changed, cols just in before, cols just in after)
- include information like diff rows (maybe just the number)
- source code of functions
- description of step, based on docstring

- nicer representation maybe in html or display or something like that

- new wrapper to include assertions and/or sanity checks (propertiy based, for example same number of mat_ids)

- function generate pipeline report, in excel (or something else)